In [68]:
from datetime import datetime, timedelta

import ccxt
import numpy as np
import pandas as pd

import plotly.graph_objects as go

ex = ccxt.binance()

In [69]:
symbols = [line.strip() for line in open("symbols.txt", "r").readlines()]

In [70]:
def get_ohlcv(symbol, days_count, timeframe):
    from_ts = int((datetime.today() - timedelta(days=days_count)).timestamp() * 1000)

    ts = []
    time = []
    open = []
    high = []
    low = []
    close = []
    volume = []

    mid = []
    open_close_dif = []
    low_high_dif = []
    open_close_dif_abs = []
    low_high_dif_abs = []

    ohlcvs = ex.fetch_ohlcv(symbol, timeframe, since=from_ts)

    for ohlcv in ohlcvs:
        t, o, h, l, c, v = ohlcv
        ts.append(t)
        time.append(pd.to_datetime(t, unit='ms'))
        open.append(o)
        high.append(h)
        low.append(l)
        close.append(c)
        volume.append(v)

        mid.append((h+l) / 2)
        open_close_dif.append(o-c)
        low_high_dif.append(l-h)
        open_close_dif_abs.append(abs(o-c))
        low_high_dif_abs.append(abs(l-h))

    return pd.DataFrame({"ts": ts, "time": time, "open": open, "high": high, "low": low, "close": close, "volume": volume, "mid": mid, "open_close_dif": open_close_dif, "low_high_dif": low_high_dif, "open_close_dif_abs": open_close_dif_abs, "low_high_dif_abs": low_high_dif_abs})

In [71]:
def fill_ohcls(ohcls, log_price = True):
    diff = np.abs(np.diff(ohcls['mid']))

    mid_shifted = ohcls['mid'][1:].to_numpy()
    mid_without_last = ohcls['mid'][:-1].to_numpy()

    if log_price:
        div = mid_shifted / mid_without_last
        log_diff = np.log(div)
    else:
        div = (mid_shifted - mid_without_last) / mid_shifted
        log_diff = div

    ohcls.loc[1:, ['diff']] = diff
    ohcls.loc[1:, ['log_diff']] = log_diff

    return ohcls[1:]

In [72]:
def plot_volatility(ohcls, annualized=False):
    volatility = go.Bar(x=ohcls['time'], y=ohcls["log_diff"], name = 'price_diff')

    time_period = ohcls.iloc[[0,-1]]['time']
    mean = ohcls['log_diff'].mean()

    y_sigma1 = annualize(ohcls) if annualized else ohcls['log_diff'].std()
    y_sigma2 = y_sigma1 * 2
    y_sigma3 = y_sigma1 * 3

    mean = go.Scatter(x=time_period, y = [mean, mean], name = 'mean', mode='lines', line={'width': 1})
    sigma1 = go.Scatter(x=time_period, y = [y_sigma1, y_sigma1], name = 'sigma1', mode='lines', line={'width': 0.5, 'color': 'lightblue'})
    sigma2 = go.Scatter(x=time_period, y = [y_sigma2, y_sigma2], name = 'sigma2', mode='lines', line={'width': 0.5, 'color': 'yellowgreen'})
    sigma3 = go.Scatter(x=time_period, y = [y_sigma3, y_sigma3], name = 'sigma3', mode='lines', line={'width': 0.5, 'color': 'magenta'})

    min_sigma1 = go.Scatter(x=time_period, y = [-y_sigma1, -y_sigma1], mode='lines', line={'width': 0.5, 'color': 'lightblue'}, showlegend=False)
    min_sigma2 = go.Scatter(x=time_period, y = [-y_sigma2, -y_sigma2], mode='lines', line={'width': 0.5, 'color': 'yellowgreen'}, showlegend=False)
    min_sigma3 = go.Scatter(x=time_period, y = [-y_sigma3, -y_sigma3], mode='lines', line={'width': 0.5, 'color': 'magenta'}, showlegend=False)

    fig = go.Figure(layout=go.Layout(
        xaxis =  {
            'showgrid': False
        },
        yaxis = {
            'showgrid': False
        }))
    fig.add_trace(volatility)
    fig.add_trace(mean)
    fig.add_trace(sigma1)
    fig.add_trace(sigma2)
    fig.add_trace(sigma3)
    fig.add_trace(min_sigma1,)
    fig.add_trace(min_sigma2)
    fig.add_trace(min_sigma3)

    fig.show()

In [73]:
def annualize(ohcls, symbol = 'None', show_period = False):
    vol = ohcls['log_diff'].std()

    timestamps = ohcls['ts'].to_numpy()
    ts_start, ts_end = timestamps[0], timestamps[-1]
    dt_ms = ts_end - ts_start
    year_ms = 365 * 24 * 60 * 60 * 1000

    n_periods = year_ms / dt_ms
    annualized = vol * np.sqrt(n_periods)

    if show_period:
        print(symbol, 'period:', int((dt_ms / 1000 / 60 / 60 / 24) * 100)/100, 'days')

    return annualized


In [74]:
def create_ohlcv_df(symbol, days_count, timeframe, log_price = True):
    """
    start volatility investigation
    :param str symbol: Unified CCXT market symbol
    :param int days_count: last days_count to fetch
    :param str timeframe: "5m","15m","30m","1h","2h","4h","6h","12h", or "1d"
    :param bool|None log_price: default True: use log(price_diff) or not
    """
    ohcls = get_ohlcv(symbol, days_count, timeframe)
    ohcls = fill_ohcls(ohcls, log_price)

    return ohcls

In [75]:
def start(symbols, days_count, timeframe, show_period = False):
    coins = []
    volatility = []

    for symbol in symbols:
        ohcls = create_ohlcv_df(symbol, days_count, timeframe)
        annualized_vol = annualize(ohcls, symbol, show_period)

        coins.append(symbol)
        volatility.append(annualized_vol)

    print(pd.DataFrame({"annual_vol": volatility}, index=coins))

In [76]:
def plot(symbol, days_count, timeframe, annualized = False):
    ohcls = create_ohlcv_df(symbol, days_count, timeframe)
    plot_volatility(ohcls, annualized)

In [77]:
start(symbols[:10], 5, '1h')

            annual_vol
FORTH/BUSD    0.046928
DOT/BUSD      0.032020
FIL/BUSD      0.037439
BTCST/BUSD    0.125455
SHIB/BUSD     0.037971
GMT/BUSD      0.038843
ICP/BUSD      0.031993
RUNE/BUSD     0.046982
APE/BUSD      0.042362
LINK/BUSD     0.043712


In [83]:
start(['LINK/BUSD'], 5, '5m', show_period=True)

LINK/BUSD period: 1.72 days
           annual_vol
LINK/BUSD    0.024189


In [84]:
start(['LINK/BUSD'], 5, '15m', show_period=True)

LINK/BUSD period: 4.97 days
           annual_vol
LINK/BUSD    0.023506


In [85]:
start(['LINK/BUSD'], 5, '1h', show_period=True)

LINK/BUSD period: 4.91 days
           annual_vol
LINK/BUSD    0.043712


In [86]:
start(['LINK/BUSD'], 5, '2h', show_period=True)

LINK/BUSD period: 4.83 days
           annual_vol
LINK/BUSD    0.061725


In [82]:
plot('BTC/USDT', 30, '1h', annualized=False)